### Graph-based SIR
To generate the node atomic models, up to a degree of N (for example, 20), run the following command from `random_graphs/nodes_atomic_generator`:

```python model_generator.py -o ../user_models -c ../../cdpp/src -m 20 && make -C ../user_models/```

This will generate the atomic sources for each kind of node, and compile CD++ with them.

In [1]:
from pringles.simulator import Simulator
from pringles.models import Coupled
from pringles.utils import VirtualTime
from pringles.display import ModelDisplay

In [2]:
simulator = Simulator(cdpp_bin_path="user_models/bin", user_models_dir="user_models/")

In [3]:
r = simulator.get_registry()

In [4]:
def build_complete_graph_of_nodes(n, exp_lambda, infection_rate):
    nodes = [r.Node1("node%d" % (i), exp_lambda=exp_lambda, infection_prob=infection_rate) for i in range(n)]
    graph = Coupled("top", nodes)
    for i in range(0,n):
        # Connecting i-th node with all others
        for j in range(0,n):
            if i != j:
                graph.add_coupling(nodes[i].get_port("out0"), nodes[j].get_port("in0"))
    return graph

In [5]:
top = build_complete_graph_of_nodes(5, 1, .5)

In [6]:
ModelDisplay(top)

In [14]:
results = simulator.run_simulation(top, VirtualTime.of_seconds(30), simulation_wd="/tmp/debugging/")

In [8]:
print(results.logs_dfs.keys())

dict_keys(['node4', 'node3', 'node2', 'node1', 'node0', 'top', 'ParallelRoot'])


In [15]:
results.logs_dfs["node4"]

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:00:745,top(06),in0,1.0,node4(05)
1,0,L,X,00:00:00:938,top(06),in0,1.0,node4(05)
2,0,L,X,00:00:01:045,top(06),in0,1.0,node4(05)
3,0,L,X,00:00:01:691,top(06),in0,1.0,node4(05)
4,0,L,X,00:00:02:691,top(06),in0,1.0,node4(05)


In [18]:
import pandas as pd

# Concatenating all nodes external transitions
nodes_external_transitions = pd.concat([results.logs_dfs[node_id] for node_id in results.logs_dfs.keys() if node_id.startswith("node")])
nodes_external_transitions = nodes_external_transitions.sort_values(by=["time"])
nodes_external_transitions

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:00:745,top(06),in0,1.0,node4(05)
0,0,L,X,00:00:00:745,top(06),in0,1.0,node3(04)
0,0,L,X,00:00:00:745,top(06),in0,1.0,node0(01)
0,0,L,X,00:00:00:745,top(06),in0,1.0,node1(02)
1,0,L,X,00:00:00:830,top(06),in0,1.0,node0(01)
1,0,L,X,00:00:00:830,top(06),in0,1.0,node3(04)
0,0,L,X,00:00:00:830,top(06),in0,1.0,node2(03)
1,0,L,X,00:00:00:830,top(06),in0,1.0,node1(02)
1,0,L,X,00:00:00:938,top(06),in0,1.0,node4(05)
2,0,L,X,00:00:00:938,top(06),in0,1.0,node0(01)
